In [1]:
%pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers.utils import logging
logging.set_verbosity(40)

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

# Загружаем модель ruGPT от сбера
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

In [4]:
text = "Как ты это делаешь?" # работает
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
out = model.generate(input_ids, do_sample=False)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



Как ты это делаешь?

— Я не знаю, — ответил он. —


# Дообучение GPT

Чтобы генерировать тексты в нужном стиле/домене, можно дообучить GPT на своем корпусе. Давайте попробуем это сделать

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Это тексты Лены Жилиной (собственные + переводы + стихотворение Бёрнса в переводе Маршака)

In [6]:
with open('/content/drive/MyDrive/ВШЭ/Магистратура/NLP/Lena_texts.txt', encoding='utf-8') as f:
    text = f.read()
text[:100]

'А что? Всего лишь третьего дня\nПопутчик славный был у меня.\nВ телегу свою усадил, в овёс\nНо никуда н'

In [7]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл
train_path = 'train_dataset.txt'
with open(train_path, "w", encoding='utf-8') as f:
    f.write(text)

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64,
                            overwrite_cache=True)

# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


## Обучение
Через класс Trainer реализовано все обучение, останется только запустить фит (точнее трейн в случае hg)

In [8]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [9]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
! pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=16,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [14]:
trainer.train()

{'train_runtime': 147.5915, 'train_samples_per_second': 59.624, 'train_steps_per_second': 0.678, 'train_loss': 0.41169525146484376, 'epoch': 100.0}


TrainOutput(global_step=100, training_loss=0.41169525146484376, metrics={'train_runtime': 147.5915, 'train_samples_per_second': 59.624, 'train_steps_per_second': 0.678, 'train_loss': 0.41169525146484376, 'epoch': 100.0})

In [16]:
text = "Как ты это делаешь?"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        temperature=0.8,
                        top_k=50,
                        max_length=300,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Как ты это делаешь? — я почти плакала. — С тобой все в порядке?

— Нет. Да, все в порядке.

— О, пожалуйста! — сказала я. — Я так по тебе скучаю! Мне так хочется, чтобы ты поскорее приехал! Я очень скучаю…

— Я приехал, чтобы увидеть тебя!

— А ты как?

— Как?

— Я бы тебя тоже увидел, если бы ты ко мне пришла…

— И я к тебе! А вы?

— Я пришел к тебе…

Он помолчал.

— Я не знаю, что мне делать, милая. А ты?

— Я не ищу тебя, — сказала я. — Я ищу тебя, чтобы знать, где ты.

Он пожал плечами.

— В любом случае, не сейчас.

И мы помолчали.

— Я в любом случае скучаю, — сказала я. — В любом случае, не сейчас.

— Ну, — сказал он, не отводя глаз от меня. — Надеюсь, ты не будешь против, если я пришлю за тобой своего человека? Я пришлю за тобой своего человека, милая.

Я опустила глаза, чтобы перевести дыхание.

— Я… я не могу, — сказала я.




Ого

In [17]:
text = "Полон город "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        temperature=0.8,
                        top_k=50,
                        max_length=300,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Полон город 
Где над лавкой, 
Сверху донизу, 
Он над лавкой, 
Сверху донизу, 
Он над лавкой, 
Сверху донизу, 
Он над лавкой...
Я жду тебя,
А ты не приходишь...
А я жду тебя...
Ты не приходишь...
Ты не приходишь...


Сонет, с которым вчера мы расстались
Мчится сквозь пространство 
Средь дней, меж тем, как я живу
Пройдет года,
И не будет в этом году
Попаданья.
И не будет в этом году
Попаданья.
Я жду тебя,
А ты не приходишь...
Ты не приходишь...
Ты не приходишь...
Будь здоров, любимый,
Счастливец,
Счастливец,
Счастливец,
Счастливец,
Счастливец,
Счастливец...

Ноябрь

Сонет, с которым вчера мы расстались
Мчится сквозь пространство
Мчится сквозь пространство
Мчится сквозь пространство
Мчится сквозь пространство
Мчится сквозь пространство
Мчится сквозь пространство
Мчится сквозь пространство
Мчится сквозь пространство
И не будет в этом году
Попаданья.
Я жду тебя, любимый,
Счастливец,
Счастливец,
Счаст


Нужно попросить не повторяться

In [18]:
text = "Полон город "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=5, top_k=50,
                        max_length=300,
                        repetition_penalty=3.5
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Полон город 
Снегом, снегом и льдом.
Ледяным вихрем
Он кружит по городу,
Прыгает с крыши на крышу,
А потом в сугробе валится.
Зелёный шарф его колышет,
И он падает замертво.
Но не тут-то было!
Как ни странно, но снег всё же капает.
На улице метёт метель,
Грустит вьюга над головой.
Вдруг кто-то из прохожих окликает:
— Эй, вы где?
Ответа нет.
Кто бы мог подумать,
Что вот так запросто может выпасть такой снег?!
Да что ж это такое?..
Вот ведь беда какая!..
Остаётся только одно — ждать…
Ждать до тех пор, пока снежный ком не собьёт тебя с ног.
Дождь льёт как из ведра,
Улицы запружены спешащими прохожими.
Они торопятся куда-то,
Чтоб хоть немного согреться.
Вода стекает со щёк,
Холод сковывает руки.
Зажмурившись от яркого солнца, они бредут по тёмному переулку.
Под ногами скрипят половицы,
По лужам скапливается грязь.
Шаги их гулко отдаются под напором ледяного ветра.
Ветер хлещет изо всех сил,
Словно загипнотизированный


Вау

Пусть будет несколько вариантов и выбор вероятного

In [22]:
text = "Приходит как-то Микеланджело в Сикстинскую капеллу. "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=6, top_k=50,
                        max_length=200,
                        repetition_penalty=3.5
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Приходит как-то Микеланджело в Сикстинскую капеллу.  И видит он, что к нему подходит какой-то человек и спрашивает: 
 - Что ты здесь делаешь? 
 - Я ищу человека по имени Микеланджело. 
 - А где же он? 
 - Он у себя в мастерской. 
 - Где же он? 
 - У себя в мастерской. 
 - Где же он? 
 - В мастерской. 
 - Где же он? 
 - На чердаке. 
 - Как это на чердаке? 
 - Да так, ничего особенного. 
 - Почему же ты не сказал мне об этом раньше? 
 - Не знаю. 
 - Но ведь я тебе скажу! 
 - Нет, не скажу. 
 - Тогда зачем же ты меня сюда притащил? 
 - Чтобы поговорить с тобой о чем-нибудь важном. 
 - О чем же? 
 - О том, что тебя мучает. 
 - Ты хочешь сказать, что кто-то


Вообще-то, он сказал: "Здрасьте", а ему: "Потолок покрасьте". Лена часто рассказывает этот анекдот.

### no_repeat_ngram_size
А этот параметр напрямую говорит, что нграммы такого размера не должны повторяться совсем

In [25]:
text = "Приходит как-то Микеланджело в Сикстинскую капеллу. "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=5, top_k=100,
                        max_length=200,
                        repetition_penalty=3.5,
                        no_repeat_ngram_size=3,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Приходит как-то Микеланджело в Сикстинскую капеллу.  Он видит, что там кто-то стоит и смотрит на него.  И он спрашивает: "Кто это?"  А тот отвечает: "Святой отец".  Так вот, этот самый святой отец говорит ему: "Я вижу, что ты стоишь и смотришь на меня.  Ты ведь знаешь, что я здесь?  Я знаю, что у меня есть для тебя кое-что...  Но не говори мне больше об этом!  Не называй меня святым отцом!" 

 Что бы вы хотели пожелать своим клиентам? 
 Чтобы ваш бизнес процветал, 
 Чтобы ваша прибыль росла, 
 Чтоб ваши доходы росли, 
 Ведь с каждым годом мы становимся все богаче! 
 Пусть сбудутся мечты ваших клиентов, 
 Пусть сбылись их заветные мечты! 
 Желаем вам крепкого здоровья, 
 Успехов во всех начинаниях, 
 Долгих лет жизни, счастья и любви! 



Ну и к чему такая концовка?.. Вообще, тексты с добавлением этого параметра часто переключались в середине на другую тему.

In [26]:
text = "Как важно быть серьёзным, "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                        do_sample=True,
                        num_beams=5, top_k=100,
                        max_length=200,
                        repetition_penalty=3.5,
                        no_repeat_ngram_size=3,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Как важно быть серьёзным, 
Чтобы не прослыть дураком.
Давай лучше о чём-нибудь другом поговорим!
О том, как я провёл лето...
А помнишь, мы с тобой гуляли по лесу?
Я помню, как ты пел мне песенку &laquo;В лесу родилась ёлочка&hellip;&raquo;.
Помню, как на поляне стояла ёлка,
И было тихо и спокойно.
Но вот в один прекрасный день кто-то постучал в её окно.
Ёлочка выглянула из окошка и увидела перед собой человека.
Он был одет в зелёный костюм, белую рубашку и шёлковую шляпу. Он подошёл к окну и спросил:
- Кто это?
- Это я, - ответила Ёлочка.
- А откуда вы знаете моё имя?
Елочка посмотрела на него широко раскрытыми глазами и смущённо потупила глаза.
Человек улыбнулся ей и сказал:


In [31]:
text = "Побывать в сказочном месте "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
for temp in [0.001, 0.1, 0.2, 0.5, 0.7, 1., 5.]:

    out = model.generate(input_ids, do_sample=True,
                     top_k=0,
                     temperature=temp,
                     max_length=50,
                     repetition_penalty=2.)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with temp - ", temp)
    print(generated_text)
    print()

### text with temp -  0.001
Побывать в сказочном месте 
И увидеть волшебный мир, 
        И услышать голос волшебной птицы.   \t\thttp://www1doma-magazin/posts_138775#

### text with temp -  0.1
Побывать в сказочном месте 
И увидеть волшебный мир, 
        И услышать голос волшебной птицы.   \t\thttp://www1domainstudio/files-2129465_pok

### text with temp -  0.2
Побывать в сказочном месте 
И увидеть волшебное чудо, 
        И услышать голос волшебной птицы.   \t\thttp://www1domainstudio/posts-2123456789

### text with temp -  0.5
Побывать в сказочном месте 
В чудесном, волшебстве. Не зря говорят: «Здесь все чудится». Но мы не знаем точно – где оно? И что это за место такое… Как поётся у Андерсена под окном

### text with temp -  0.7
Побывать в сказочном месте 
Без лишних слов, без суеты. И остаться с тобою навсегда! 
   О чём говорить? Что делать и где искать ответы… А что бы ты хотел узнать о мире живых?.. Расскажи мне всё своё детство

### text with temp -  1.0
Побывать в сказочном 

В общем, мне 0.5-0.7 нравится